In [1]:
import os

In [2]:
%pwd

'e:\\Text_summarization\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'e:\\Text_summarization'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    data_path: Path
    model_ckpt: Path
    num_train_epochs: int
    warmup_steps: int
    per_device_train_batch_size: int
    weight_decay: float
    logging_steps: int
    evaluation_strategy: str
    eval_steps: int
    save_steps: float
    gradient_accumulation_steps: int

In [6]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.TrainingArguments

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            model_ckpt = config.model_ckpt,
            num_train_epochs = params.num_train_epochs,
            warmup_steps = params.warmup_steps,
            per_device_train_batch_size = params.per_device_train_batch_size,
            weight_decay = params.weight_decay,
            logging_steps = params.logging_steps,
            evaluation_strategy = params.evaluation_strategy,
            eval_steps = params.evaluation_strategy,
            save_steps = params.save_steps,
            gradient_accumulation_steps = params.gradient_accumulation_steps
        )

        return model_trainer_config

In [8]:
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorForSeq2Seq
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from datasets import load_dataset, load_from_disk
import torch

[2025-01-22 14:56:14,864: INFO: config: PyTorch version 2.5.1 available.]


In [9]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Trainer, TrainingArguments, DataCollatorForSeq2Seq
import torch
import os
from datasets import load_from_disk

class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    def train(self):
        device = "cuda" if torch.cuda.is_available() else "cpu"
        
        # Load T5 model and tokenizer
        tokenizer = AutoTokenizer.from_pretrained("t5-small")  # T5 tokenizer
        model_t5 = AutoModelForSeq2SeqLM.from_pretrained("t5-small").to(device)
        seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_t5)

        # Load dataset (assuming it's stored in the correct format)
        dataset_samsum_pt = load_from_disk(self.config.data_path)
        
        # Select a larger sample for training
        train_dataset = dataset_samsum_pt["train"].select(range(100))
        eval_dataset = dataset_samsum_pt["validation"].select(range(100))  # Use the validation split

        # Tokenize the dataset
        def tokenize_function(examples):
            # Ensure input and labels are tokenized with padding and truncation
            model_inputs = tokenizer(
                examples["dialogue"],
                max_length=1024,
                padding="max_length",
                truncation=True,
            )
            
            # Tokenize summaries as labels
            with tokenizer.as_target_tokenizer():
                labels = tokenizer(
                    examples["summary"], 
                    max_length=128, 
                    padding="max_length", 
                    truncation=True
                )
                
            model_inputs["labels"] = labels["input_ids"]
            return model_inputs

        # Apply tokenization
        tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True)
        tokenized_eval_dataset = eval_dataset.map(tokenize_function, batched=True)
        
        # Set parameters for training
        trainer_args = TrainingArguments(
            output_dir=self.config.root_dir,  # where the model checkpoints will be saved
            num_train_epochs=3,  # Increased epochs
            warmup_steps=500,  # Increase warmup steps for smoother learning
            per_device_train_batch_size=2,  # Increased batch size to 2
            per_device_eval_batch_size=2,  # Increased eval batch size to 2
            weight_decay=0.01,  # Small weight decay for regularization
            logging_steps=10,  # Log every 10 steps to reduce log frequency
            evaluation_strategy='steps',  # Perform evaluation every `eval_steps`
            eval_steps=500,  # Perform evaluation every 500 steps
            save_steps=500,  # Save checkpoints every 500 steps
            gradient_accumulation_steps=2,  # Accumulate gradients over 2 steps
            fp16=False,  # Disable mixed precision (don't need for low-end systems)
            load_best_model_at_end=False,  # Don't worry about the best model
            logging_dir='./logs',  # Optional, directory to store logs
            report_to=[]  # Disable reporting to external systems like TensorBoard
        )

        # Initialize Trainer
        trainer = Trainer(
            model=model_t5,
            args=trainer_args,
            tokenizer=tokenizer,
            data_collator=seq2seq_data_collator,
            train_dataset=tokenized_train_dataset,  # Use tokenized train dataset
            eval_dataset=tokenized_eval_dataset  # Use tokenized eval dataset for evaluation
        )
        
        # Start training
        trainer.train()

        # Save the model and tokenizer (optional)
        model_t5.save_pretrained(os.path.join(self.config.root_dir, "t5-samsum-model"))
        tokenizer.save_pretrained(os.path.join(self.config.root_dir, "tokenizer"))


In [10]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise e

[2025-01-22 14:56:16,447: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-01-22 14:56:16,450: INFO: common: yaml file: params.yaml loaded successfully]
[2025-01-22 14:56:16,452: INFO: common: created directory at: artifacts]
[2025-01-22 14:56:16,455: INFO: common: created directory at: artifacts/model_trainer]


Map:   0%|          | 0/100 [00:00<?, ? examples/s]

c:\Users\ADITYA\anaconda3\envs\TextSummarization\Lib\site-packages\transformers\tokenization_utils_base.py:3961: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
c:\Users\ADITYA\anaconda3\envs\TextSummarization\Lib\site-packages\transformers\training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
C:\Users\ADITYA\AppData\Local\Temp\ipykernel_7864\4286148607.py:71: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. 

Step,Training Loss,Validation Loss
